In [81]:
import graphlab as gl

#needed for charts
import matplotlib.pyplot as plt
%matplotlib inline

# Loading Data

In [82]:
# loading the train and test data
customers = gl.SFrame('train.csv/train.csv')
test_sframe = gl.SFrame('test.csv/test.csv')

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

Parsing completed. Parsed 100 lines in 2.4477 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,float,float,int,int,float,float,float,int,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,int,float,float,int,int,int,int,int,float,int,float,float,float,float,int,int,int,float,float,int,int,int,float,float,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,i

Read 67150 lines. Lines per second: 19434.8

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

...

Parsing completed. Parsed 100 lines in 2.00914 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,int,int,int,int,int,float,float,float,float,float,float,float,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,int,float,float,float,int,int,float,int,int,int,int,float,float,float,int,int,int,float,float,float,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int

Read 67325 lines. Lines per second: 18456.3

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/test.csv/test.csv

Parsing completed. Parsed 75818 lines in 3.95382 secs.

# Exploring the Data

In [83]:
#gl.canvas.set_target('ipynb')
#customers['TARGET'].show(view='Categorical')
#customers.show()

# Pre-processing

## Removing constant columns

In [84]:
#remove all columns with constant columns; where STD == 0
remove = []

for col in customers.column_names():
    if customers[col].std() == 0:
        remove.append(col)

customers.remove_columns(remove)
customers.num_columns()

337

## Removing duplicated columns (too slow!!!)

## Add sum of 0's

In [85]:
# select on the features, leaving out the 'TARGET' feature
features = [f for f in customers.column_names() if f != 'TARGET']

## NOTE: Causes overfitting (need to study why)
# add the sum of 0's
#customers['sum_of_0'] = customers[features].apply(lambda x: len([i for i in x.values() if i == 0]))

## Remove low variance features

In [86]:
# it did improve the results a little -> 0.811884 -> 0.812585
# These features were spitted with low variance from the classifier
low_variance_features = ['delta_imp_reemb_var33_1y3', 'delta_num_reemb_var33_1y3', 'imp_reemb_var17_hace3', 'imp_reemb_var33_ult1', 'imp_trasp_var17_in_hace3', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in_hace3']
customers.remove_columns(low_variance_features)
customers.num_columns()

329

### Most Target are 0, which means most customers are satisfied

In [87]:
train_data, test_data = customers.random_split(.8, seed=2)

## Train a Boosted Trees Classifier

In [167]:
customer_model = gl.boosted_trees_classifier.create(train_data, target='TARGET', row_subsample=0.7, 
                                                    max_depth=5, metric='auc',
                                                   column_subsample=0.8)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 57748

Number of classes           : 2

...

| 4         | 1.841679     | 0.829191     | 0.865689       |

| 5         | 2.554363     | 0.837843     | 0.873600       |

| 6         | 3.092317     | 0.841092     | 0.873282       |

| 10        | 5.038424     | 0.855678     | 0.883464       |

+-----------+--------------+--------------+----------------+

## Train a Deep Learning Network

In [168]:
customer_model.evaluate(test_data)

{'accuracy': 0.962157915411811,
 'auc': 0.831680948811279,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 3
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  577  |
 |      0       |        0        | 14696 |
 |      0       |        1        |   1   |
 +--------------+-----------------+-------+
 [3 rows x 3 columns],
 'f1_score': 0.0,
 'log_loss': 0.14103486663622974,
 'precision': 0.0,
 'recall': 0.0,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-------+
 | threshold | fpr | tpr |  p  |   n   |
 +-----------+-----+-----+-----+-------+
 |    0.0    | 1.0 | 1.0 | 577 | 14697 |
 |   1e-05   | 1.0 | 1.0 | 577 | 14697 |
 |   2e-05   | 1.0 | 1.0 | 577 | 14697 |
 |   3e-05   | 1.0 | 1.0 | 577 | 14697 |
 |   4e-05   | 1.0 | 1.0 | 

## Save Predictions

In [161]:
prediction = customer_model.predict(test_sframe, output_type='probability')

In [162]:
#function for making submission
def make_submission(prediction, filename='submission.txt'):
    with open(filename, 'w') as f:
        #header
        f.write('ID,TARGET\n')
        submission_strings = test_sframe['ID'].astype(str) + ',' + prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

make_submission(prediction, 'submission11.txt')


In [92]:
'''
# removing columns that are giving the same information
import numpy as np
remove_duplicates = []
c = customers.column_names()
#test = customers['TARGET'].to_numpy()
for i in range(len(c) - 1):
    v = customers[c[i]].to_numpy()
    for j in range(i+1, len(c)):
        if np.array_equal(v, customers[c[j]].to_numpy()):
            remove_duplicates.append(c[j])
    print "round", i, "done"
print remove
#giving the same information ???


#net = gl.deeplearning.create(train_data, target='TARGET')
net = gl.deeplearning.MultiLayerPerceptrons(num_hidden_layers=4,
                                           num_hidden_units=[32,64,32,2],
                                           activation='relu',
                                           learning_rate=0.001,
                                           max_iterations=5000)
                                           
dl_customer_model = gl.neuralnet_classifier.create(train_data,'TARGET',
                                                  network=net,
                                                  valiation_data=test_data)

'''

'\n# removing columns that are giving the same information\nimport numpy as np\nremove_duplicates = []\nc = customers.column_names()\n#test = customers[\'TARGET\'].to_numpy()\nfor i in range(len(c) - 1):\n    v = customers[c[i]].to_numpy()\n    for j in range(i+1, len(c)):\n        if np.array_equal(v, customers[c[j]].to_numpy()):\n            remove_duplicates.append(c[j])\n    print "round", i, "done"\nprint remove\n#giving the same information ???\n'